# 准备工作

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pylab as pl
import time
import datetime
from datetime import timedelta
import calendar
from tqdm import tqdm
from sklearn import preprocessing
from mpl_toolkits.mplot3d import Axes3D

In [2]:
import math
import re

# 数据获取

In [4]:
#小技巧：排除数据源中的干扰行
def fliter_none(issue):
    pattern = re.compile('\d{6}.[A-Z]{2}')
    try:
        id_stock = pattern.search(issue)
        if id_stock is not None :
            return True
        else:
            return False
    except:
        return False 

## 股票

In [27]:
#导入股票信息数据
stock_filepath = './data/A股信息_202012.xlsx'
all_stock = pd.read_excel(stock_filepath,sheet_name='share')
all_stock.iloc[:,2:] = all_stock.iloc[:,2:].astype(float)
# all_stock['market_value'] = all_stock['market_value']/100000000                   #变成以亿元为单位的市值数据
all_stock = all_stock.sort_values('market_value',axis = 0,ascending=0)             #按照市值降序排列
all_stock['cum_market_value'] = all_stock['market_value'].cumsum(axis=0)
all_stock['Cum_Ratio'] = all_stock['cum_market_value']/sum(all_stock['market_value'].dropna())  #计算累计市值百分比

all_stock

,stock_code,stock_name,share_num,close_price,market_value,PE,PB,PR,PC,dp,...,g3e,g3b,g3r,g3c,g_e,g_b,g_r,g_c,cum_market_value,Cum_Ratio
2833,600519.SH,贵州茅台,1.256198e+09,1998.00,2.509883e+12,56.3023,16.9070,27.3676,58.3609,0.9416,...,0.299629,0.233862,0.264884,0.868149,0.201151,0.208440,0.178679,0.367812,2.509883e+12,0.029713
3318,601398.SH,工商银行,3.564063e+11,4.99,1.712803e+12,6.1499,0.6892,2.0351,1.3275,5.1056,...,0.037975,0.099200,0.065100,-0.060613,0.028918,0.093324,0.067476,0.380387,4.222686e+12,0.049990
3302,601318.SH,中国平安,1.828024e+10,86.98,1.537702e+12,12.9395,2.2405,1.3326,8.6948,3.1442,...,0.206413,0.175674,0.096478,0.701471,0.188854,0.172699,0.111791,0.390929,5.760388e+12,0.068195
3409,601939.SH,建设银行,2.500110e+11,6.28,1.252059e+12,6.3509,0.7128,2.1291,1.5484,4.7042,...,0.041667,0.115702,0.059883,6.687500,0.033730,0.100423,0.067340,2.210121,7.012448e+12,0.083017
349,000858.SZ,五粮液,3.881608e+09,291.85,1.132847e+12,58.3819,14.1083,20.4083,104.2732,0.8521,...,0.363422,0.164104,0.296832,0.233403,0.266881,0.162643,0.230851,0.248580,8.145295e+12,0.096428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4300,688685.SH,迈信林,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4146,...,0.080000,0.102632,0.723754,-0.642857,0.408709,0.221187,0.370864,3.579157,NaN,NaN
4302,688687.SH,凯因科技,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2292,...,0.608696,0.075425,0.438914,3.166667,0.379598,0.097493,0.198060,1.353498,NaN,NaN
4303,688689.SH,银河微电,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6667,...,0.017544,0.135834,-0.043021,-0.069444,0.089592,0.112791,0.003354,0.091237,NaN,NaN
4304,688696.SH,极米科技,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4131,...,-0.423313,0.034295,0.660825,4.503788,1.059383,-0.083057,0.113636,0.827492,NaN,NaN


## 基金

In [7]:
#导入基金持仓明细数据
fund_filepath = './data/基金全部持股(明细)_202012.xlsx'
all_fund = pd.read_excel(fund_filepath)
all_fund = all_fund[all_fund['代码'].apply(fliter_none)]  #排除数据源中的干扰行,最后是“数据来源wind”
all_fund

,代码,名称,报告期,股票代码,股票简称,持股数量(万股),半年度持股变动(万股),持股占流通股比(%),持股市值(万元),占净值比(%),占股票投资市值比(%),投资类型,管理公司
0,000001.OF,华夏成长,2020年年报,601318.SH,中国平安,333.7526,NaN,0.030810,29029.801148,6.035095,8.016716,偏股混合型基金,华夏基金管理有限公司
1,000001.OF,华夏成长,2020年年报,002127.SZ,南极电商,2074.4721,166.6814,1.057654,28378.778328,5.899752,7.836933,偏股混合型基金,华夏基金管理有限公司
2,000001.OF,华夏成长,2020年年报,000961.SZ,中南建设,2889.7307,NaN,0.759232,25516.322081,5.304667,7.046452,偏股混合型基金,华夏基金管理有限公司
3,000001.OF,华夏成长,2020年年报,600519.SH,贵州茅台,12.0575,8.4349,0.009598,24090.885000,5.008328,6.652811,偏股混合型基金,华夏基金管理有限公司
4,000001.OF,华夏成长,2020年年报,002271.SZ,东方雨虹,566.8427,18.3244,0.337064,21993.496760,4.572296,6.073608,偏股混合型基金,华夏基金管理有限公司
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19806,000841.OF,富国新回报AB,2020年年报,300882.SZ,万胜智能,0.0440,NaN,0.001180,1.134760,0.001751,0.002043,灵活配置型基金,富国基金管理有限公司
19807,000841.OF,富国新回报AB,2020年年报,605277.SH,新亚电子,0.0507,NaN,0.001520,0.859365,0.001326,0.001547,灵活配置型基金,富国基金管理有限公司
19808,000841.OF,富国新回报AB,2020年年报,003030.SZ,祖名股份,0.0480,NaN,0.001538,0.728640,0.001124,0.001312,灵活配置型基金,富国基金管理有限公司
19809,000841.OF,富国新回报AB,2020年年报,300886.SZ,华业香料,0.0147,NaN,0.001080,0.671349,0.001036,0.001209,灵活配置型基金,富国基金管理有限公司


# 计算xy得分

## 股票

In [143]:
## 大中小盘股票门限值
LMT = all_stock['market_value'].quantile(0.3)                           #大中盘门限值
MST = all_stock['market_value'].quantile(0.1)                           #中小盘门限值


In [158]:
#计算VCG
def calculate_VCG(stock_code):
    multi = all_stock[all_stock['stock_code']==stock_code][['ep','bp','rp','cp','dp']]   #获取该股票的估值参数
    growth = all_stock[all_stock['stock_code']==stock_code][['g_e','g_b','g_r','g_c']]   #获取该股票的盈利增长参数
    
    
    OVS = 0.5 * multi['ep'] + 0.125 * (multi['bp']+multi['rp']+multi['cp']+multi['dp'])  #计算股票价值得分
    OGS = 0.25 * (sum(growth.iloc[0,:]))                                                  #计算股票成长得分
    VCG = OGS - OVS                                                         #成长得分-价值得分    
    return VCG.values[0]
###注意：VCG返回值是一个复合类型。加“.values[0]”！！！

for stock_code in all_stock['stock_code']:
    all_stock.loc[all_stock['stock_code']==stock_code,'VCG'] = calculate_VCG(stock_code)

###求大中小盘各类股票中的风格类型门限值
#大盘股
big_stock = all_stock[all_stock['market_value']>LMT]
VT_big = big_stock['VCG'].quantile(0.3333)
GT_big = big_stock['VCG'].quantile(0.6666)
#中盘股
mid_stock = all_stock[(all_stock['market_value']>=MST)&(all_stock['market_value']<LMT)]
VT_mid = mid_stock['VCG'].quantile(0.3333)
GT_mid = mid_stock['VCG'].quantile(0.6666)
#小盘股
small_stock = all_stock[all_stock['market_value']<MST]
VT_small = small_stock['VCG'].quantile(0.3333)
GT_small = small_stock['VCG'].quantile(0.6666)



In [159]:
#门限值汇总
threshold ={'LMT':LMT,'MST':MST,
              'VT_big':VT_big,'GT_big':GT_big,
              'VT_mid':VT_mid,'GT_mid':GT_mid,
              'VT_small':VT_small,'GT_small':GT_small}
threshold

{'LMT': 3542277587.2560005,
 'MST': 2121584958.6,
 'VT_big': -0.42224034302961405,
 'GT_big': -0.017515544471351407,
 'VT_mid': -0.4592353559734895,
 'GT_mid': -0.11195411242579453,
 'VT_small': -0.5156495102402098,
 'GT_small': -0.1718822060529751}

In [219]:
#计算每只股票市值得分——y轴得分
def calculate_score_y_stock(stock_code):
    cap = all_stock[all_stock['stock_code']==stock_code]['market_value']    #该股票市值
    cap = cap.values[0]  #cap是一个复合类型
    if np.isnan(cap):          
        return "股票市值数据缺失"
    else:
        score_y_stock = 100*(1+(math.log(cap)-math.log(MST))/(math.log(LMT)-math.log(MST)))
        return score_y_stock
         

In [223]:
#计算组内股票价值成长得分——x轴得分
def calculate_score_x_stock(stock_code):
    VCG = calculate_VCG(stock_code)
    if np.isnan(VCG):
        return "VCG计算数据缺失"
    else:
        flag = identify_stock_size(stock_code)       #判断该股票所属规模类型，不同类型的阈值不同
        if flag == '大盘':   
            VT = VT_big
            GT = GT_big
        else:
            if flag == '中盘':
                VT = VT_mid
                GT = GT_mid
            else:
                VT = VT_small
                GT = GT_small
            
        score_x_stock = 100*(1+(VCG-VT)/(GT-VT))
        return score_x_stock

## 基金

In [285]:
#计算持仓股票加权市值得分——y轴得分
def calculate_score_y_fund(fund_code):
    holdings = all_fund[all_fund['代码']==fund_code][['股票代码','占股票投资市值比(%)']]
    for stock_code in np.array(holdings['股票代码']):
        holdings.loc[holdings['股票代码']==stock_code,'该股票y得分'] = calculate_score_y_stock(stock_code)  ## 给dataframe增加一列
        nocap = holdings[holdings['该股票y得分']=="股票市值数据缺失"]['股票代码'].values #市值数据缺失的股票列表
    if len(nocap) !=0:  
        print("有股票市值数据缺失，其股票代码如下：")
        print(nocap)
        return False
    else:
        score_y_fund = np.inner(np.array(holdings['该股票y得分']),np.array(holdings['占股票投资市值比(%)']))*0.01
        return score_y_fund

In [286]:
#计算持仓股票加权价值成长得分——x轴得分
def calculate_score_x_fund(fund_code):
    holdings = all_fund[all_fund['代码']==fund_code][['股票代码','占股票投资市值比(%)']]
    for stock_code in np.array(holdings['股票代码']):
        holdings.loc[holdings['股票代码']==stock_code,'该股票x得分'] = calculate_score_x_stock(stock_code) ##
    novcg = holdings[holdings['该股票x得分']=="VCG计算数据缺失"]['股票代码'].values #VCG计算数据缺失的股票列表
    if len(novcg) !=0:  
        print("有股票VCG计算数据缺失，其股票代码如下：")
        print(novcg)
        return False
    else:
        score_x_fund = np.inner(np.array(holdings['该股票x得分']),np.array(holdings['占股票投资市值比(%)']))*0.01
        return score_x_fund

# 类型判断

## 股票

In [271]:
#根据市值得分划分规模类型（股票）
def identify_stock_size(stock_code):
    score_y_stock = calculate_score_y_stock(stock_code)
    if score_y_stock == "股票市值数据缺失":
        return "该股票市值数据缺失,无法判断规模类型"
    else:
        if score_y_stock > 200:
            return '大盘'
        else:
            if score_y_stock >=100:
                return '中盘'
            else:
                return '小盘'

In [272]:
#根据价值成长得分判断风格类型（股票）
def identify_stock_style(stock_code):    
    score_x_stock = calculate_score_x_stock(stock_code)
    if score_x_stock == "VCG计算数据缺失":
        return "该股票VCG计算数据缺失,无法判断风格类型"
    else:
        if score_x_stock > 200:
            return '成长型'
        else:
            if score_x_stock >=100:
                return '混合型'
            else:
                return '价值型'       

## 基金

In [279]:
#根据市值得分划分规模类型（基金）
def identify_fund_size(fund_code):
    score_y_fund =calculate_score_y_fund(fund_code)
    if score_y_fund == False:
        print("因为数据缺失无法判断规模类型")
    else:
        if score_y_fund >200:
            return '大盘'
        else:
            if score_y_fund >=100:
                return '中盘'
            else:
                return '小盘'


In [280]:
#根据价值成长得分判断风格类型（基金）
def identify_fund_style(fund_code,gamma=0.5):
    score_x_fund =calculate_score_x_fund(fund_code)
    if score_x_fund == False:
        print("因为数据缺失无法判断风格类型")
    else:
        if score_x_fund > 150*(1+gamma/3):
            return '成长型'
        else:
            if score_x_fund >=150*(1-gamma/3):
                return '混合型'
            else:
                return '价值型'

# 案例

## 股票

### 数据不缺失

In [241]:
#中国平安
stock_code1 = '601318.SH'

In [244]:
multi = all_stock[all_stock['stock_code']==stock_code1][['ep','bp','rp','cp','dp']]   #获取该股票的估值参数
growth = all_stock[all_stock['stock_code']==stock_code1][['g_e','g_b','g_r','g_c']]            #获取该股票的盈利增长参数
OVS = 0.5 * multi['ep'] + 0.125 * (multi['bp']+multi['rp']+multi['cp']+multi['dp'])  #计算股票价值得分
OGS = 0.25 * (sum(growth.iloc[0,:]))                                                       #计算股票成长得分
VCG = OGS - OVS
VCG

###注意：VCG是一个复合类型。
####calculate_VCG函数返回值需要加“.values[0]”！！！


3302   -0.379567
dtype: float64

In [296]:
all_stock.loc[all_stock['stock_code'] == stock_code1,['market_value','ep','bp','rp','cp','dp','g_e','g_b','g_r','g_c']]

,market_value,ep,bp,rp,cp,dp,g_e,g_b,g_r,g_c
3302,1.537702e+12,0.077283,0.446329,0.750413,0.115011,3.1442,0.188854,0.172699,0.111791,0.390929


In [297]:
calculate_VCG(stock_code1)

-0.3795673158723164

In [298]:
calculate_score_x_stock(stock_code1)  

110.5437144719845

In [299]:
calculate_score_y_stock(stock_code1)

1384.7830342204975

In [300]:
identify_stock_size(stock_code1)

'大盘'

In [301]:
identify_stock_style(stock_code1)

'混合型'

### 数据缺失

In [233]:
##新亚电子
stock_code2 = '605277.SH'

In [291]:
all_stock.loc[all_stock['stock_code'] == stock_code2,['market_value','ep','bp','rp','cp','dp','g_e','g_b','g_r','g_c']]

,market_value,ep,bp,rp,cp,dp,g_e,g_b,g_r,g_c
4003,NaN,NaN,NaN,NaN,NaN,1.4416,NaN,0.529968,-0.043499,0.046094


In [293]:
calculate_score_y_stock(stock_code2)

'股票市值数据缺失'

In [235]:
calculate_score_x_stock(stock_code2)

'VCG计算数据缺失'

In [295]:
identify_stock_size(stock_code2)

'该股票市值数据缺失,无法判断规模类型'

In [237]:
identify_stock_style(stock_code2)

'VCG计算数据缺失,无法判断风格类型'

## 基金

### 数据不缺失

In [331]:
### 宝盈祥瑞A
fund_code2 = '000639.OF'
all_fund[all_fund['代码']==fund_code2] #持仓详情

,代码,名称,报告期,股票代码,股票简称,持股数量(万股),半年度持股变动(万股),持股占流通股比(%),持股市值(万元),占净值比(%),占股票投资市值比(%),投资类型,管理公司
15127,000639.OF,宝盈祥瑞A,2020年年报,600585.SH,海螺水泥,0.65,NaN,0.000163,33.5530,3.249200,16.428801,偏债混合型基金,宝盈基金管理有限公司
15128,000639.OF,宝盈祥瑞A,2020年年报,600900.SH,长江电力,1.60,NaN,0.000070,30.6560,2.968661,15.010322,偏债混合型基金,宝盈基金管理有限公司
15129,000639.OF,宝盈祥瑞A,2020年年报,600547.SH,山东黄金,1.10,NaN,0.000334,25.9820,2.516041,12.721757,偏债混合型基金,宝盈基金管理有限公司
15130,000639.OF,宝盈祥瑞A,2020年年报,000651.SZ,格力电器,0.28,NaN,0.000047,17.3432,1.679478,8.491878,偏债混合型基金,宝盈基金管理有限公司
15131,000639.OF,宝盈祥瑞A,2020年年报,601088.SH,中国神华,0.80,NaN,0.000049,14.4080,1.395239,7.054694,偏债混合型基金,宝盈基金管理有限公司
15132,000639.OF,宝盈祥瑞A,2020年年报,300463.SZ,迈克生物,0.25,NaN,0.000636,11.6500,1.128161,5.704275,偏债混合型基金,宝盈基金管理有限公司
15133,000639.OF,宝盈祥瑞A,2020年年报,600309.SH,万华化学,0.10,NaN,0.000070,9.1040,0.881612,4.457658,偏债混合型基金,宝盈基金管理有限公司
15134,000639.OF,宝盈祥瑞A,2020年年报,000568.SZ,泸州老窖,0.04,NaN,0.000027,9.0464,0.876034,4.429455,偏债混合型基金,宝盈基金管理有限公司
15135,000639.OF,宝盈祥瑞A,2020年年报,300136.SZ,信维通信,0.25,NaN,0.000306,8.9700,0.868635,4.392047,偏债混合型基金,宝盈基金管理有限公司
15136,000639.OF,宝盈祥瑞A,2020年年报,600887.SH,伊利股份,0.20,NaN,0.000034,8.8740,0.859339,4.345042,偏债混合型基金,宝盈基金管理有限公司


In [332]:
calculate_score_y_fund(fund_code2)

941.7958812287194

In [333]:
calculate_score_x_fund(fund_code2)

108.4322182082493

In [334]:
identify_fund_size(fund_code2)

'大盘'

In [335]:
identify_fund_style(fund_code2)

'价值型'

### 数据缺失

In [322]:
#华夏大盘成长
fund_code1 = '000011.OF'
all_fund[all_fund['代码']==fund_code1] #持仓详情

,代码,名称,报告期,股票代码,股票简称,持股数量(万股),半年度持股变动(万股),持股占流通股比(%),持股市值(万元),占净值比(%),占股票投资市值比(%),投资类型,管理公司
787,000011.OF,华夏大盘精选,2020年年报,600754.SH,锦江酒店,1083.6716,44.8251,1.351319,55841.597548,7.822949,8.547976,偏股混合型基金,华夏基金管理有限公司
788,000011.OF,华夏大盘精选,2020年年报,600690.SH,海尔智家,1517.0146,117.6077,0.240470,44311.996466,6.207746,6.783077,偏股混合型基金,华夏基金管理有限公司
789,000011.OF,华夏大盘精选,2020年年报,603806.SH,福斯特,459.1191,-101.0303,0.596605,39208.771140,5.492827,6.001899,偏股混合型基金,华夏基金管理有限公司
790,000011.OF,华夏大盘精选,2020年年报,002311.SZ,海大集团,598.2614,153.2784,0.363353,39186.121700,5.489654,5.998432,偏股混合型基金,华夏基金管理有限公司
791,000011.OF,华夏大盘精选,2020年年报,601021.SH,春秋航空,691.5670,299.9868,0.754605,38333.558810,5.370216,5.867925,偏股混合型基金,华夏基金管理有限公司
...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,000011.OF,华夏大盘精选,2020年年报,300917.SZ,特发服务,0.0281,NaN,0.001179,0.897514,0.000126,0.000137,偏股混合型基金,华夏基金管理有限公司
864,000011.OF,华夏大盘精选,2020年年报,605277.SH,新亚电子,0.0507,NaN,0.001520,0.859365,0.000120,0.000132,偏股混合型基金,华夏基金管理有限公司
865,000011.OF,华夏大盘精选,2020年年报,003030.SZ,祖名股份,0.0480,NaN,0.001538,0.728640,0.000102,0.000112,偏股混合型基金,华夏基金管理有限公司
866,000011.OF,华夏大盘精选,2020年年报,300886.SZ,华业香料,0.0147,NaN,0.001080,0.671349,0.000094,0.000103,偏股混合型基金,华夏基金管理有限公司


In [287]:
calculate_score_y_fund(fund_code1)

有股票市值数据缺失，其股票代码如下：
['688617.SH' '300926.SZ' '300927.SZ' '605277.SH' '003030.SZ' '003031.SZ']


False

In [288]:
calculate_score_x_fund(fund_code)

有股票VCG计算数据缺失，其股票代码如下：
['601021.SH' '600258.SH' '000876.SZ' '600009.SH' '300999.SZ' '688617.SH'
 '300926.SZ' '300919.SZ' '300927.SZ' '605277.SH' '003030.SZ' '003031.SZ']


False

In [289]:
identify_fund_size(fund_code1)

有股票市值数据缺失，其股票代码如下：
['688617.SH' '300926.SZ' '300927.SZ' '605277.SH' '003030.SZ' '003031.SZ']
因为数据缺失无法判断规模类型


In [290]:
identify_fund_style(fund_code1)

有股票VCG计算数据缺失，其股票代码如下：
['601021.SH' '600258.SH' '000876.SZ' '600009.SH' '300999.SZ' '688617.SH'
 '300926.SZ' '300919.SZ' '300927.SZ' '605277.SH' '003030.SZ' '003031.SZ']
因为数据缺失无法判断风格类型
